#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире - ( в доме мб ? )

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [5]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [319]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


ну тут сепаратор потребовалось добавить, классика. Добавляю его и двигаемся дальше, взглянув, что такое вообще есть у нас с помощью head()

In [8]:
df = pd.read_csv('input_data.csv', sep=';')

In [10]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


In [12]:
df.describe()

,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
count,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.135815e+07,1.085038e+07,7.152596e+06,1.135815e+07,8.096207e+06
mean,6.787516e+06,6.426675e+00,1.176266e+01,1.719417e+00,5.310356e+01,-2.674071e+00,5.440394e+01,5.229570e+01,1.017820e+00,5.275366e-01,4.031717e+05,3.525623e+05,5.125515e+01,1.703792e+06
std,1.977118e+08,5.283144e+00,7.218441e+00,1.157606e+00,2.712845e+01,3.241855e+01,4.582962e+00,2.183899e+01,1.562077e+00,8.813503e-01,1.959618e+05,1.366485e+05,2.250527e+01,6.404578e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,1.000000e+00,-1.000000e+02,4.143758e+01,-1.732189e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.161870e+05,1.000000e+00,5.886060e+05
25%,2.600000e+06,2.000000e+00,5.000000e+00,1.000000e+00,3.650000e+01,0.000000e+00,5.319411e+01,3.763847e+01,0.000000e+00,0.000000e+00,1.961580e+05,2.339150e+05,3.100000e+01,1.162465e+06
50%,3.995000e+06,5.000000e+00,1.000000e+01,2.000000e+00,4.670000e+01,6.500000e+00,5.564688e+01,4.202111e+01,0.000000e+00,0.000000e+00,4.000070e+05,3.502220e+05,5.400000e+01,1.703143e+06
75%,6.500000e+06,9.000000e+00,1.700000e+01,2.000000e+00,6.300000e+01,1.050000e+01,5.687362e+01,6.547322e+01,2.000000e+00,2.000000e+00,6.250070e+05,4.705370e+05,7.200000e+01,2.255297e+06
max,6.355524e+11,5.000000e+01,5.000000e+01,9.000000e+00,4.999000e+02,4.080000e+02,7.350889e+01,1.792583e+02,6.000000e+00,2.000000e+00,8.621630e+05,5.885970e+05,2.000000e+02,2.839173e+06


In [14]:
df.info

<bound method DataFrame.info of                 date     price  level  levels  rooms  area  kitchen_area  \
0         2021-01-01   2451300     15      31      1  30.3           0.0   
1         2021-01-01   1450000      5       5      1  33.0           6.0   
2         2021-01-01  10700000      4      13      3  85.0          12.0   
3         2021-01-01   3100000      3       5      3  82.0           9.0   
4         2021-01-01   2500000      2       3      1  30.0           9.0   
...              ...       ...    ...     ...    ...   ...           ...   
11358145  2021-12-31   6099000      4       9      3  65.0           0.0   
11358146  2021-12-31   2490000      1      10      2  56.9           0.0   
11358147  2021-12-31    850000      2       2      2  37.0           5.0   
11358148  2021-12-31   4360000      5       5      1  36.0           9.0   
11358149  2021-12-31   1850000      1       5      1  33.0           7.0   

            geo_lat    geo_lon  building_type  object_t

Отобразим на карте координаты наших построек.

In [17]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

Вау! Никогда не видел подобной визуализации на карте с такой точностью с помощью OSM, как раз по беспилотным автомобилям что-то может подобное понадобится в проекте, огонь !

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

так, ну тут следует аккуратно прикинуть сначала координаты, которые я буду считать за центры Москвы и Питера, и потом по радиусу окружности использовать понимание 20 км и смотреть, попадает ли объект в эту окружность

In [19]:
MOSCOW_CENTER = (55.7522, 37.6156) # широта, долгота, соответственно, Lat, Lon - Москвы
PITER_CENTER = (59.9343, 30.3351) # широта, долгота, соответственно, Lat, Lon - Санкт-Петербурга

формула хаверсина достаточно интересная, надо будет делать всякие переводы в радианы, в нампай-векторы. Сначала найти угловое расстояние между точками на сфере ( ну, наша планета Земля будет видимо ) , а потом перевеси в линейное ( км ) умножением на Радиус Земли. И так уже смотрим, меньше ли будет 20 км

In [21]:
def haversine_distance(lat1, lon1, lat2, lon2, R=6371.0032): # радиус взял средний
    lat1 = np.asarray(lat1, dtype=float)
    lon1 = np.asarray(lon1, dtype=float)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    raznphi = np.radians(lat2 - lat1)
    raznlmb = np.radians(lon2 - lon1)
    a = np.sin(raznphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(raznlmb/2.0)**2
    return 2 * R * np.arcsin(np.sqrt(a))

после функции уже надо добавить два признака в наш датасет, где будет соответственно 1 - если в центре ( в 20 км от уентра ) от Москвы, иначе - 0; И идиентично с Питером

In [23]:
MSC_DIST = haversine_distance(df["geo_lat"], df["geo_lon"], MOSCOW_CENTER[0], MOSCOW_CENTER[1])
PITER_DIST = haversine_distance(df["geo_lat"], df["geo_lon"], PITER_CENTER[0], PITER_CENTER[1])

In [24]:
df["is_Moscow"]          = (MSC_DIST <= 20).astype(int)
df["is_Saint_Peterburg"] = (PITER_DIST    <= 20).astype(int)

И хочется по примеру выше красивому, когда делали пометки на карте - сделать также пометку, проверить, верно ли я сделал вообще задау - но уже с домами, которые наодятся в центре Москвы, Питера

In [28]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,0,0


In [30]:
dfv = df.dropna(subset=["geo_lat", "geo_lon"])

msk = dfv[dfv["is_Moscow"] == 1]
spb = dfv[dfv["is_Saint_Peterburg"] == 1]

msk500 = msk.sample(n=min(500, len(msk)), random_state=42)
spb500 = spb.sample(n=min(500, len(spb)), random_state=42)

m = folium.Map(location=[57.0, 40.0], zoom_start=5)

fg_msk = folium.FeatureGroup(name=f"Moscow (n={len(msk500)})").add_to(m)
fg_spb = folium.FeatureGroup(name=f"Saint-Petersburg (n={len(spb500)})").add_to(m)


for lat, lon in zip(msk500["geo_lat"], msk500["geo_lon"]):
    folium.Marker([lat, lon], icon=folium.Icon(color="red")).add_to(fg_msk)

for lat, lon in zip(spb500["geo_lat"], spb500["geo_lon"]):
    folium.Marker([lat, lon], icon=folium.Icon(color="blue")).add_to(fg_spb)

folium.LayerControl().add_to(m)

bounds = []
for subset in (msk500, spb500):
    if len(subset):
        bounds.append([subset["geo_lat"].min(), subset["geo_lon"].min()])
        bounds.append([subset["geo_lat"].max(), subset["geo_lon"].max()])
if bounds:
    m.fit_bounds(bounds)

display(m)


видим, что реально близко соответсвенно к Москве, к Питеру - работаем дальше !

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [32]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,0,0


In [34]:
df = df.drop(columns=["geo_lat", "geo_lon", "object_type", "postal_code", "street_id", "id_region", "house_id"])
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: date

Числовые: price, level, levels, rooms, area, kitchen_area, building_type, is_Moscow, is_Saint_Peterburg

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

с помощью info() функции удобно взглянуть, какие вообще типы у нас. И видим, что все являются числовыми, кроме первого - date

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   price               int64  
 2   level               int64  
 3   levels              int64  
 4   rooms               int64  
 5   area                float64
 6   kitchen_area        float64
 7   building_type       int64  
 8   is_Moscow           int32  
 9   is_Saint_Peterburg  int32  
dtypes: float64(2), int32(2), int64(5), object(1)
memory usage: 779.9+ MB


In [38]:
df['date'].nunique()

365

у нас значит 365, видимо в течение года смотрели квартиры, дальше надо их закодировать в  категориальные с помоощью oneHot-Encoding

In [40]:
from sklearn.preprocessing import OneHotEncoder

In [42]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output=False).set_output(transform='pandas')

In [44]:
ohetransform = ohe.fit_transform(df[['date']])

Вижу, что выдает ошибку, так как очень много у нас материала, пишет, что весит около 30 гб, это жестко. Попробую через разреженный вариант ниже, где буду на каждое число ( 0 или 1 ) тратить всего 8 бит, чтобы максимально экономно обойтись!

In [47]:
df["date"] = df["date"].astype("category")
is_new_api = tuple(map(int, sklearn.__version__.split(".")[:2])) >= (1, 2)
ohe = OneHotEncoder(handle_unknown="ignore", dtype=np.int8, **({"sparse_output": True} if is_new_api else {"sparse": True}))
date_sparse = ohe.fit_transform(df[["date"]])
date_cols = ohe.get_feature_names_out(["date"])

mat = date_sparse.tocsc()
step = 32
for s in range(0, mat.shape[1], step):
    sub = mat[:, s:s+step]
    sub_df = pd.DataFrame.sparse.from_spmatrix(sub, index=df.index, columns=date_cols[s:s+step])
    df = pd.concat([df, sub_df], axis=1, copy=False)


In [49]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg,...,date_2021-12-22,date_2021-12-23,date_2021-12-24,date_2021-12-25,date_2021-12-26,date_2021-12-27,date_2021-12-28,date_2021-12-29,date_2021-12-30,date_2021-12-31
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


сработало! ОГОНЬ!

In [52]:
df['date_2021-12-22'].sum()

32087

In [54]:
df['date_2021-12-27'].sum()

23946

In [56]:
df['date_2021-01-01'].sum()

7984

In [58]:
df.shape

(11358150, 375)

чето меня насторожило, что у date_2021-01-01 так много единиц стоит вряд - сверху проверил, что рандомные из остальных не нулевые в сумме своей

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [69]:
d = pd.to_datetime(df["date"].astype("string"), format="%Y-%m-%d", errors="coerce")
min_dt = d.min()

In [73]:
df["days_since_first"] = (d - min_dt).dt.days.astype("int32")

In [75]:
df["floor_ratio"] = (df["level"].astype("float32") / df["levels"].replace(0, np.nan).astype("float32")).clip(0, 1).fillna(0).astype("float32")


In [77]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg,...,date_2021-12-24,date_2021-12-25,date_2021-12-26,date_2021-12-27,date_2021-12-28,date_2021-12-29,date_2021-12-30,date_2021-12-31,floor_ratio,days_since_first
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0.483871,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0,...,0,0,0,0,0,0,0,0,1.000000,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0,...,0,0,0,0,0,0,0,0,0.307692,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0,...,0,0,0,0,0,0,0,0,0.600000,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0,...,0,0,0,0,0,0,0,0,0.666667,0


вижу, что в конце реально добавились нужные колонки, по второй точно видно, что нули стоят, так как после первой ничего не прошло еще как бы. Также можно попробовать максимальное найти в конце, должно выйти 364 - как проверка просто

In [82]:
df["days_since_first"].max()

364

да, все сходится, все четко !

чуть не забыл, надо еще отнормиовать через Standart Scaler !

для начала удалю дату:

In [90]:
df = df.drop(columns=["date"])

In [91]:
df.head()

,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg,date_2021-01-01,...,date_2021-12-24,date_2021-12-25,date_2021-12-26,date_2021-12-27,date_2021-12-28,date_2021-12-29,date_2021-12-30,date_2021-12-31,floor_ratio,days_since_first
0,2451300,15,31,1,30.3,0.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0.483871,0
1,1450000,5,5,1,33.0,6.0,0,0,0,1,...,0,0,0,0,0,0,0,0,1.000000,0
2,10700000,4,13,3,85.0,12.0,3,0,0,1,...,0,0,0,0,0,0,0,0,0.307692,0
3,3100000,3,5,3,82.0,9.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0.600000,0
4,2500000,2,3,1,30.0,9.0,3,0,0,1,...,0,0,0,0,0,0,0,0,0.666667,0


один умный товарищ подсказал, что не следует скейлить OHE, таргет, и бинарки, будем этому следовать

In [95]:
from sklearn.preprocessing import StandardScaler

In [97]:
TARGET = "price"

In [99]:
date_ohe_cols = [c for c in df.columns if c.startswith("date_")]
binary_cols   = [c for c in df.select_dtypes(include=["number","bool"]).columns if df[c].nunique(dropna=False) <= 2]

In [101]:
dont_scale    = set(date_ohe_cols + binary_cols + ["building_type", TARGET])

In [103]:
num_all  = df.select_dtypes(include=["number"]).columns.tolist()
to_scale = [c for c in num_all if c not in dont_scale]

In [105]:
scaler_all = StandardScaler()
df.loc[:, to_scale] = scaler_all.fit_transform(df[to_scale]).astype("float32")
print("Scaled columns:", len(to_scale))

C:\Users\fgrac\AppData\Local\Temp\ipykernel_15996\3540468225.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.6227695  -0.27004272 -0.45932394 ... -0.8378864  -0.27004272
 -1.0271676 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, to_scale] = scaler_all.fit_transform(df[to_scale]).astype("float32")
C:\Users\fgrac\AppData\Local\Temp\ipykernel_15996\3540468225.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2.6650274  -0.93685853  0.17141403 ... -1.3524607  -0.93685853
 -0.93685853]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, to_scale] = scaler_all.fit_transform(df[to_scale]).astype("float32")
C:\Users\fgrac\AppData\Local\Temp\ipykernel_15996\3540468225.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated 

Scaled columns: 7


In [107]:
df.head()

,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg,date_2021-01-01,...,date_2021-12-24,date_2021-12-25,date_2021-12-26,date_2021-12-27,date_2021-12-28,date_2021-12-29,date_2021-12-30,date_2021-12-31,floor_ratio,days_since_first
0,2451300,1.622769,2.665027,-0.621470,-0.840577,0.082486,0,0,0,1,...,0,0,0,0,0,0,0,0,-0.310650,-1.858355
1,1450000,-0.270043,-0.936859,-0.621470,-0.741051,0.267565,0,0,0,1,...,0,0,0,0,0,0,0,0,1.468654,-1.858355
2,10700000,-0.459324,0.171414,1.106234,1.175756,0.452644,3,0,0,1,...,0,0,0,0,0,0,0,0,-0.918009,-1.858355
3,3100000,-0.648605,-0.936859,1.106234,1.065171,0.360105,0,0,0,1,...,0,0,0,0,0,0,0,0,0.089693,-1.858355
4,2500000,-0.837886,-1.213927,-0.621470,-0.851636,0.360105,3,0,0,1,...,0,0,0,0,0,0,0,0,0.319520,-1.858355


отстандартировалось, огнище !!!

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [121]:
import numpy as np

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = int(n_neighbors)
        if metric != 'euclidean':
            raise ValueError("Only 'euclidean' is supported.")
        self.metric = metric
        self._X = None
        self._y = None
        self._X_sq = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).ravel()
        if X.shape[0] != y.shape[0]:
            raise ValueError("X and y must have the same number of rows.")
        self._X = X
        self._y = y
        self._X_sq = np.einsum('ij,ij->i', X, X)  # ||x||^2 per row
        return self

    def predict(self, X):
        if self._X is None:
            raise RuntimeError("Call fit() first.")
        X = np.asarray(X, dtype=float)
        k = max(1, min(self.n_neighbors, self._X.shape[0]))
        A_sq = np.einsum('ij,ij->i', X, X)[:, None]
        cross = X @ self._X.T
        D2 = A_sq + self._X_sq[None, :] - 2.0 * cross
        np.maximum(D2, 0.0, out=D2)
        D = np.sqrt(D2)
        idx = np.argpartition(D, kth=k-1, axis=1)[:, :k]
        return self._y[idx].mean(axis=1)


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [126]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.lr   = float(learning_rate)
        self.opt  = optimization.upper()
        self.eps  = float(epsilon)   # для AdaGrad
        self.beta = float(decay_rate) # для Momentum
        self.max_iter = int(max_iter)
        self.weights = None
        self.bias = 0.0

    def fit(self, X, y):
        X = np.asarray(X, float)
        y = np.asarray(y, float).ravel()
        n, d = X.shape
        self.weights = np.zeros(d, float)
        self.bias = 0.0

        v_w = np.zeros(d, float); v_b = 0.0   # Momentum
        G_w = np.zeros(d, float); G_b = 0.0   # AdaGrad

        for _ in range(self.max_iter):
            yhat = X @ self.weights + self.bias
            err  = yhat - y
            grad_w = (2.0/n) * (X.T @ err)
            grad_b = (2.0/n) * err.sum()

            if self.opt == 'SGD':
                self.weights -= self.lr * grad_w
                self.bias    -= self.lr * grad_b
            elif self.opt == 'MOMENTUM':
                v_w = self.beta * v_w + grad_w
                v_b = self.beta * v_b + grad_b
                self.weights -= self.lr * v_w
                self.bias    -= self.lr * v_b
            elif self.opt == 'ADAGRAD':
                G_w += grad_w**2
                G_b += grad_b**2
                self.weights -= (self.lr / (np.sqrt(G_w) + self.eps)) * grad_w
                self.bias    -= (self.lr / (np.sqrt(G_b) + self.eps)) * grad_b
            else:
                raise ValueError("optimization must be 'SGD', 'Momentum', or 'AdaGrad'")
        return self

    def predict(self, X):
        X = np.asarray(X, float)
        return X @ self.weights + self.bias


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [ ]:
# ===== 1) train/test 80/20 =====
from sklearn.model_selection import train_test_split
X = df.drop(columns=[TARGET])
y = df[TARGET] 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# ===== 2) обучение моделей =====
from sklearn.neighbors import KNeighborsRegressor as SKKNN
from sklearn.linear_model import LinearRegression as SKLinear

USE_SAMPLE = True
SAMPLE_TRAIN = 200_000
SAMPLE_TEST  = 50_000

if USE_SAMPLE:
    tr_idx = X_train.sample(min(SAMPLE_TRAIN, len(X_train)), random_state=42).index
    te_idx = X_test.sample(min(SAMPLE_TEST,  len(X_test)),  random_state=42).index
else:
    tr_idx = X_train.index
    te_idx = X_test.index


Xtr_np = X_train.loc[tr_idx].to_numpy(dtype=float)
ytr_np = y_train.loc[tr_idx].to_numpy(dtype=float)
Xte_np = X_test.loc[te_idx].to_numpy(dtype=float)
yte_np = y_test.loc[te_idx].to_numpy(dtype=float)

knn_my = KNNRegressor(n_neighbors=7).fit(Xtr_np, ytr_np)
lin_my = LinearRegression(learning_rate=0.01, optimization='ADAGRAD', max_iter=60)
lin_my.fit(Xtr_np, ytr_np)


knn_sk = SKKNN(n_neighbors=7).fit(X_train.to_numpy(float), y_train.to_numpy(float))
lin_sk = SKLinear().fit(X_train.to_numpy(float), y_train.to_numpy(float))

# ===== 3) метрики + сравнение =====
import numpy as np, pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

def mse(y, p):  return mean_squared_error(y, p)
def mae(y, p):  return mean_absolute_error(y, p)
def rmse(y, p): return sqrt(mean_squared_error(y, p))

preds = {
    "KNN (ours)":                knn_my.predict(Xte_np),
    "Linear (ours, AdaGrad)":    lin_my.predict(Xte_np),
    "sklearn KNN":               knn_sk.predict(X_test.to_numpy(float)),
    "sklearn LinearRegression":  lin_sk.predict(X_test.to_numpy(float)),
}

rows = []
for name, p in preds.items():
    if "ours" in name:
        yt = yte_np
    else:
        yt = y_test.to_numpy(float)
    rows.append({
        "model": name,
        "MSE_test":  mse(yt, p),
        "MAE_test":  mae(yt, p),
        "RMSE_test": rmse(yt, p),
        "R2_test":   1.0 - (np.sum((yt - p)**2) / np.sum((yt - yt.mean())**2)),
        "n_eval":    len(yt)
    })
metrics_df = pd.DataFrame(rows).sort_values("RMSE_test").reset_index(drop=True)
display(metrics_df)